In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals

In [ ]:
from typing import Any

import pandas as pd
import polars as pl

import narwhals as nw

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [ ]:
Q_NUM = 21

IO_FUNCS = {
    "pandas": lambda x: pd.read_parquet(x, engine="pyarrow"),
    "pandas[pyarrow]": lambda x: pd.read_parquet(
        x, engine="pyarrow", dtype_backend="pyarrow"
    ),
    "polars[eager]": lambda x: pl.read_parquet(x),
    "polars[lazy]": lambda x: pl.scan_parquet(x),
}

In [ ]:
def q21(
    lineitem_raw: Any,
    nation_raw: Any,
    orders_raw: Any,
    supplier_raw: Any,
) -> Any:
    lineitem = nw.from_native(lineitem_raw)
    nation = nw.from_native(nation_raw)
    orders = nw.from_native(orders_raw)
    supplier = nw.from_native(supplier_raw)

    var1 = "SAUDI ARABIA"

    q1 = (
        lineitem.group_by("l_orderkey")
        #         .agg(nw.col("l_suppkey").len().alias("n_supp_by_order"))
        .agg(nw.len().alias("n_supp_by_order"))
        .filter(nw.col("n_supp_by_order") > 1)
        .join(
            lineitem.filter(nw.col("l_receiptdate") > nw.col("l_commitdate")),
            #             on="l_orderkey",
            left_on="l_orderkey",
            right_on="l_orderkey",
        )
    )

    q_final = (
        q1.group_by("l_orderkey")
        #         .agg(nw.col("l_suppkey").len().alias("n_supp_by_order"))
        .agg(nw.len().alias("n_supp_by_order"))
        .join(
            q1,
            left_on="l_orderkey",
            right_on="l_orderkey",
            # on="l_orderkey"
        )
        .join(supplier, left_on="l_suppkey", right_on="s_suppkey")
        .join(nation, left_on="s_nationkey", right_on="n_nationkey")
        .join(orders, left_on="l_orderkey", right_on="o_orderkey")
        .filter(nw.col("n_supp_by_order") == 1)
        .filter(nw.col("n_name") == var1)
        .filter(nw.col("o_orderstatus") == "F")
        .group_by("s_name")
        .agg(nw.len().alias("numwait"))
        .sort(by=["numwait", "s_name"], descending=[True, False])
        .head(100)
    )
    return nw.to_native(q_final)

In [ ]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"

lineitem = dir_ + "lineitem.parquet"
nation = dir_ + "nation.parquet"
orders = dir_ + "orders.parquet"
supplier = dir_ + "supplier.parquet"

In [ ]:
results = {}

## pandas via Narwhals

In [ ]:
tool = "pandas"
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = (
    fn(lineitem),
    fn(nation),
    fn(orders),
    fn(supplier),
)

timings = %timeit -o -q q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, via Narwhals

In [ ]:
tool = "pandas[pyarrow]"
fn = IO_FUNCS[tool]
lineitem_raw, nation_raw, orders_raw, supplier_raw = (
    fn(lineitem),
    fn(nation),
    fn(orders),
    fn(supplier),
)

timings = %timeit -o -q q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = "polars[eager]"
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = (
    fn(lineitem),
    fn(nation),
    fn(orders),
    fn(supplier),
)
timings = %timeit -o -q q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = "polars[lazy]"
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = (
    fn(lineitem),
    fn(nation),
    fn(orders),
    fn(supplier),
)
timings = %timeit -o -q q21(lineitem_raw, nation_raw, orders_raw, supplier_raw).collect()
results[tool] = timings.all_runs

## Save

In [ ]:
import json

with open("results.json", "w") as fd:
    json.dump(results, fd)

In [ ]:
from pprint import pprint

pprint(results)